In [1]:
import tensorflow as tf
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt

In [2]:
#biomag_labeled_1 = sio.loadmat('data_labeled_5.mat')
biomag_labeled_1 = sio.loadmat(r"D:\python_project\wip\data_labeled_for_py_1_2.mat")

trX, trY, teX, teY, vaX, vaY = biomag_labeled_1['x_train'], biomag_labeled_1['y_train'],\
                               biomag_labeled_1['y_test'], biomag_labeled_1['x_test'],\
                               biomag_labeled_1['x_val'], biomag_labeled_1['y_val'] 

epochs = 50
alpha = 1.0
batchsize = 100

momentum = 0.001

In [3]:
sio.whosmat(r"D:\python_project\wip\data_labeled_for_py_1_2.mat")

[('x_train', (2094, 254), 'double'),
 ('y_train', (1, 2094), 'double'),
 ('x_test', (206, 254), 'double'),
 ('y_test', (206, 1), 'uint8'),
 ('x_val', (317, 254), 'double'),
 ('y_val', (317, 12), 'uint8')]

In [4]:
trY=np.reshape(trY,2094)
trY.shape

(2094,)

In [5]:
X = tf.placeholder("float", [None, 254])
Y = tf.placeholder("float", [None, 12])

rbm_w = tf.placeholder("float", [254, 500])
rbm_vb = tf.placeholder("float", [254])
rbm_hb = tf.placeholder("float", [500])

h0 = tf.nn.sigmoid(tf.matmul(X, rbm_w) + rbm_hb)
v1 = tf.nn.sigmoid(tf.matmul(h0, tf.transpose(rbm_w)) + rbm_vb)
h1 = tf.nn.sigmoid(tf.matmul(v1, rbm_w) + rbm_hb)

In [6]:
#grad calculation
w_positive_grad = tf.matmul(tf.transpose(X), h0)
w_negative_grad = tf.matmul(tf.transpose(v1), h1)

update_w = rbm_w + alpha * (w_positive_grad - w_negative_grad) / tf.to_float(tf.shape(X)[0])
update_vb = rbm_vb + alpha * tf.reduce_mean(X - v1, 0)
update_hb = rbm_hb + alpha * tf.reduce_mean(h0 - h1, 0)

h_sample = tf.nn.sigmoid(tf.matmul(X, rbm_w) + rbm_hb)
v_sample = tf.nn.sigmoid(tf.matmul(h_sample, tf.transpose(rbm_w)) + rbm_vb)

#define MSE
err = X - v_sample
err_sum = tf.reduce_mean(err * err)

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [7]:
#init weights and biases
n_w = np.zeros([254, 500], np.float32)
n_vb = np.zeros([254], np.float32)
n_hb = np.zeros([500], np.float32)
o_w = np.zeros([254, 500], np.float32)
o_vb = np.zeros([254], np.float32)
o_hb = np.zeros([500], np.float32)

#training
for e in range(epochs):
    print(sess.run(err_sum, feed_dict={X: trX, rbm_w: o_w, rbm_vb: o_vb, rbm_hb: o_hb}))
    for start, end in zip(range(0, len(trX), batchsize), range(batchsize, len(trX), batchsize)):
        batch = trX[start:end]
        n_w = sess.run(update_w, feed_dict={X: batch, rbm_w: o_w, rbm_vb: o_vb, rbm_hb: o_hb})
        n_vb = sess.run(update_vb, feed_dict={X: batch, rbm_w: o_w, rbm_vb: o_vb, rbm_hb: o_hb})
        n_hb = sess.run(update_hb, feed_dict={X: batch, rbm_w: o_w, rbm_vb: o_vb, rbm_hb: o_hb})
        o_w = n_w
        o_vb = n_vb
        o_hb = n_hb
        if start % 2000 == 0:
            print("MSE: ", sess.run(err_sum, feed_dict={X: trX, rbm_w: n_w, rbm_vb: n_vb, rbm_hb: n_hb}))


0.124251075
MSE:  0.074030735
0.018301835
MSE:  0.018232886


0.01798589
MSE:  0.017976513
0.017960126
MSE:  0.017953912
0.0179548
MSE:  0.017949054


0.017953089
MSE:  0.017947452
0.017952353
MSE:  0.017946748
0.017951949
MSE: 

 0.017946353
0.017951684
MSE:  0.017946092
0.017951485
MSE:  0.017945891


0.01795132
MSE:  0.017945727
0.017951177
MSE:  0.017945582
0.017951053
MSE:  0.017945454


...

0.017949414
MSE:  0.017943777
0.01794937
MSE:  0.017943736
0.017949332
MSE:  0.017943693


0.01794929
MSE:  0.017943652
0.01794925
MSE:  0.017943608
0.017949207
MSE:  0.017943565


0.017949166
MSE:  0.017943522
0.017949123
MSE:  0.017943477
0.017949082


MSE:  0.017943434
0.017949037
MSE:  0.01794339
0.01794899
MSE: 

 0.017943345


#2nd layer
---------------------------------------------------------------------------------------------

In [8]:
rbm_w2 = tf.placeholder("float", [500, 500])
rbm_vb2 = tf.placeholder("float", [500])
rbm_hb2 = tf.placeholder("float", [500])

h2 = tf.nn.sigmoid(tf.matmul(h0, rbm_w2) + rbm_hb2)
v2 = tf.nn.sigmoid(tf.matmul(h2, tf.transpose(rbm_w2)) + rbm_vb2)
h3 = tf.nn.sigmoid(tf.matmul(v2, rbm_w2) + rbm_hb2)

#grad calculation
w_positive_grad = tf.matmul(tf.transpose(h0), h2)
w_negative_grad = tf.matmul(tf.transpose(v2), h3)

update_w = rbm_w2 + alpha * (w_positive_grad - w_negative_grad) / tf.to_float(tf.shape(h1)[0])
update_vb = rbm_vb2 + alpha * tf.reduce_mean(h0 - v2, 0)
update_hb = rbm_hb2 + alpha * tf.reduce_mean(h2 - h3, 0)

h_sample = tf.nn.sigmoid(tf.matmul(h0, rbm_w2) + rbm_hb2)
v_sample = tf.nn.sigmoid(tf.matmul(h_sample, tf.transpose(rbm_w2)) + rbm_vb2)

#define MSE
#X = tf.placeholder("float", [None, 784])          #error dimension fix  -s -k  | X-et kivenni
err = h0 - v_sample
err_sum = tf.reduce_mean(err * err)

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

n_w_2 = np.zeros([500, 500], np.float32)
n_vb_2 = np.zeros([500], np.float32)
n_hb_2 = np.zeros([500], np.float32)
o_w_2 = np.zeros([500, 500], np.float32)
o_vb_2 = np.zeros([500], np.float32)
o_hb_2 = np.zeros([500], np.float32)


#training
print(sess.run(err_sum, feed_dict={rbm_w2: o_w_2, rbm_vb2: o_vb_2, rbm_hb2: o_hb_2, X: trX, rbm_hb : o_hb, rbm_w: o_w}))
for start, end in zip(range(0, len(trX), batchsize), range(batchsize, len(trX), batchsize)):
    batch = trX[start:end]
    n_w_2 = sess.run(update_w, feed_dict={X: batch, rbm_w2: o_w_2, rbm_vb2: o_vb_2, rbm_hb2: o_hb_2,  rbm_hb : o_hb, rbm_w: o_w,
                                          rbm_vb: o_vb})
    n_vb_2 = sess.run(update_vb, feed_dict={X: batch, rbm_w2: o_w_2, rbm_vb2: o_vb_2, rbm_hb2: o_hb_2,  rbm_hb : o_hb, rbm_w: o_w,
                                            rbm_vb: o_vb})
    n_hb_2 = sess.run(update_hb, feed_dict={X: batch, rbm_w2: o_w_2, rbm_vb2: o_vb_2, rbm_hb2: o_hb_2,  rbm_hb : o_hb, rbm_w: o_w,
                                            rbm_vb: o_vb})
    o_w_2 = n_w_2
    o_vb_2 = n_vb_2
    o_hb_2 = n_hb_2
    if start % 1000 == 0:
        print("MSE: ", sess.run(err_sum, feed_dict={rbm_w2: o_w_2, rbm_vb2: o_vb_2, rbm_hb2: o_hb_2, X: trX, rbm_hb : o_hb, rbm_w: o_w}))



0.24987753
MSE:  6.3008365e-08


MSE:  6.3008365e-08


#3nd layer
---------------------------------------------------------------------------------------------

In [9]:
rbm_w3 = tf.placeholder("float", [500, 500])
rbm_vb3 = tf.placeholder("float", [500])
rbm_hb3 = tf.placeholder("float", [500])

h4 = tf.nn.sigmoid(tf.matmul(h2, rbm_w3) + rbm_hb3)
v3 = tf.nn.sigmoid(tf.matmul(h4, tf.transpose(rbm_w3)) + rbm_vb3)
h5 = tf.nn.sigmoid(tf.matmul(v3, rbm_w3) + rbm_hb3)

#grad calculation
w_positive_grad = tf.matmul(tf.transpose(h2), h4)
w_negative_grad = tf.matmul(tf.transpose(v3), h5)

update_w = rbm_w3 + alpha * (w_positive_grad - w_negative_grad) / tf.to_float(tf.shape(h2)[0])
update_vb = rbm_vb3 + alpha * tf.reduce_mean(h2 - v3, 0)
update_hb = rbm_hb3 + alpha * tf.reduce_mean(h4 - h5, 0)

h_sample = tf.nn.sigmoid(tf.matmul(h2, rbm_w3) + rbm_hb3)
v_sample = tf.nn.sigmoid(tf.matmul(h_sample, tf.transpose(rbm_w3)) + rbm_vb3)

#define MSE
#X = tf.placeholder("float", [None, 784])          #error dimension fix  -s -k  | X-et kivenni
err = h2 - v_sample
err_sum = tf.reduce_mean(err * err)

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

n_w_3 = np.zeros([500, 500], np.float32)
n_vb_3 = np.zeros([500], np.float32)
n_hb_3 = np.zeros([500], np.float32)
o_w_3 = np.zeros([500, 500], np.float32)
o_vb_3 = np.zeros([500], np.float32)
o_hb_3 = np.zeros([500], np.float32)


#training
print(sess.run(err_sum, feed_dict={rbm_w3: o_w_3, rbm_vb3: o_vb_3, rbm_hb3: o_hb_3, X: trX,
                                   rbm_hb2 : o_hb_2, rbm_w2: o_w_2,
                                   rbm_hb : o_hb, rbm_w: o_w, rbm_vb: o_vb}))
for start, end in zip(range(0, len(trX), batchsize), range(batchsize, len(trX), batchsize)):
    batch = trX[start:end]
    n_w_3 = sess.run(update_w, feed_dict={X: batch, rbm_w3: o_w_3, rbm_vb3: o_vb_3, rbm_hb3: o_hb_3,
                                          rbm_w2: o_w_2, rbm_vb2: o_vb_2, rbm_hb2: o_hb_2,
                                          rbm_hb : o_hb, rbm_w: o_w, rbm_vb: o_vb})
    n_vb_3 = sess.run(update_vb, feed_dict={X: batch, rbm_w3: o_w_3, rbm_vb3: o_vb_3, rbm_hb3: o_hb_3,
                                          rbm_w2: o_w_2, rbm_vb2: o_vb_2, rbm_hb2: o_hb_2,
                                          rbm_hb : o_hb, rbm_w: o_w, rbm_vb: o_vb})
    n_hb_3 = sess.run(update_hb, feed_dict={X: batch, rbm_w3: o_w_3, rbm_vb3: o_vb_3, rbm_hb3: o_hb_3,
                                          rbm_w2: o_w_2, rbm_vb2: o_vb_2, rbm_hb2: o_hb_2,
                                          rbm_hb : o_hb, rbm_w: o_w, rbm_vb: o_vb})
    o_w_3 = n_w_3
    o_vb_3 = n_vb_3
    o_hb_3 = n_hb_3
    if start % 1000 == 0:
        print("MSE: ", sess.run(err_sum, feed_dict={rbm_w3: o_w_3, rbm_vb3: o_vb_3, rbm_hb3: o_hb_3, X: trX,
                                                    rbm_w2: o_w_2, rbm_vb2: o_vb_2, rbm_hb2: o_hb_2,
                                                    rbm_hb : o_hb, rbm_w: o_w}))



0.0005170225
MSE:  0.004894533
MSE:  1.5767986e-05


In [10]:
#Classification---------------------------------------------------------------------------------------------
#weigths and biases for new output layer
out_w = tf.Variable(tf.random_normal([500, 12]))
out_b = tf.Variable(tf.random_normal([12]))
learning_rate = 0.01

# #reformat y train labels
trY_formazott = np.zeros([biomag_labeled_1['y_train'].shape[1],12])
# #biomag_labeled_1['y_test']
for rows in range((biomag_labeled_1['y_train'].shape[1])):
    #rows +=1
    current_class_label = biomag_labeled_1['y_train'][0,rows] 
    current_class_label = int(current_class_label)
    trY_formazott[rows,current_class_label -1] = 1
    # if biomag_labeled_1['y_train'][rows] != 0:
    #     trY_formazott[rows] = columns+1 #trY_formazando[rows,columns]
trY = trY_formazott




In [12]:
#trY[567]
trY.shape
#trY_formazott[4]

(2094, 12)

In [13]:
#network definition
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.nn.sigmoid(tf.matmul(x, o_w) + o_hb)
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_1, out_w) + out_b
    return out_layer

# Construct model
logits = neural_net(X)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init2 = tf.global_variables_initializer()
# Run the initializer
sess.run(init2)




#biomag_labeled_1['y_test']
#Y
#trY[355]
#biomag_labeled_1['y_train'][2,0]
#trY_formazott = np.zeros([biomag_labeled_1['y_train'].shape[0],12])
#Y
#trY_formazott[0]

In [14]:
#Training top ANN layer
for epochs_2 in range(10):
    print("Epoch ",epochs_2 +1)
    for start, end in zip(range(0, len(trX), batchsize), range(batchsize, len(trX), batchsize)):
        batch_x = trX[start:end]
        batch_y = trY[start:end]
        
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if start % 500 == 0:
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x, Y: batch_y})
            print("Step " + str(start/batchsize) + ", Minibatch Loss= " + \
              "{:.4f}".format(loss) + ", Training Accuracy= " + \
              "{:.3f}".format(acc))


#batch_x.shape
#trY_formazott.shape

Epoch  1
Step 0.0, Minibatch Loss= 2.9835, Training Accuracy= 0.140
Step 5.0, Minibatch Loss= 3.0265, Training Accuracy= 0.090
Step 10.0, Minibatch Loss= 2.8655, Training Accuracy= 0.120
Step 15.0, Minibatch Loss= 2.8604, Training Accuracy= 0.120
Epoch  2
Step 0.0, Minibatch Loss= 2.7696, Training Accuracy= 0.140
Step 5.0, Minibatch Loss= 2.8041, Training Accuracy= 0.090
Step 10.0, Minibatch Loss= 2.6939, Training Accuracy= 0.120


Step 15.0, Minibatch Loss= 2.6762, Training Accuracy= 0.120
Epoch  3
Step 0.0, Minibatch Loss= 2.5808, Training Accuracy= 0.140
Step 5.0, Minibatch Loss= 2.6031, Training Accuracy= 0.090
Step 10.0, Minibatch Loss= 2.5412, Training Accuracy= 0.110
Step 15.0, Minibatch Loss= 2.5084, Training Accuracy= 0.120


Epoch  4
Step 0.0, Minibatch Loss= 2.4159, Training Accuracy= 0.140
Step 5.0, Minibatch Loss= 2.4211, Training Accuracy= 0.090
Step 10.0, Minibatch Loss= 2.4063, Training Accuracy= 0.110
Step 15.0, Minibatch Loss= 2.3568, Training Accuracy= 0.120
Epoch 

 5
Step 0.0, Minibatch Loss= 2.2733, Training Accuracy= 0.140
Step 5.0, Minibatch Loss= 2.2580, Training Accuracy= 0.090
Step 10.0, Minibatch Loss= 2.2904, Training Accuracy= 0.110
Step 15.0, Minibatch Loss= 2.2243, Training Accuracy= 0.110
Epoch  6
Step 0.0, Minibatch Loss= 2.1535, Training Accuracy= 0.130
Step 5.0, Minibatch Loss= 2.1154, Training Accuracy= 0.080
Step 10.0, Minibatch Loss= 2.1944, Training Accuracy= 0.110


Step 15.0, Minibatch Loss= 2.1130, Training Accuracy= 0.110
Epoch  7
Step 0.0, Minibatch Loss= 2.0566, Training Accuracy= 0.120
Step 5.0, Minibatch Loss= 1.9947, Training Accuracy= 0.080
Step 10.0, Minibatch Loss= 2.1181, Training Accuracy= 0.090
Step 15.0, Minibatch Loss= 2.0238, Training Accuracy= 0.110


Epoch  8
Step 0.0, Minibatch Loss= 1.9817, Training Accuracy= 0.140
Step 5.0, Minibatch Loss= 1.8968, Training Accuracy= 0.120
Step 10.0, Minibatch Loss= 2.0605, Training Accuracy= 0.370
Step 15.0, Minibatch Loss= 1.9559, Training Accuracy= 0.500
Epoch  9
Step 0.0, Minibatch Loss= 1.9265, Training Accuracy= 0.460
Step 5.0, Minibatch Loss= 1.8206, Training Accuracy= 0.570
Step 10.0, Minibatch Loss= 2.0189, Training Accuracy= 0.450
Step 15.0, Minibatch Loss= 1.9066, Training Accuracy= 0.500
Epoch  10
Step 0.0, Minibatch Loss= 1.8873, Training Accuracy= 0.460


Step 5.0, Minibatch Loss= 1.7634, Training Accuracy= 0.570
Step 10.0, Minibatch Loss= 1.9899, Training Accuracy= 0.450
Step 15.0, Minibatch Loss= 1.8723, Training Accuracy= 0.500


In [19]:
#biomag_labeled_1['y_train'].shape[0]
# Evaluate the Model
# Define the input function for evaluating
input_fn_test = tf.estimator.inputs.numpy_input_fn(
    x={'val_data_x': vaX}, y=vaY,
    batch_size=batchsize, shuffle=False)
# Use the Estimator 'evaluate' method
model.evaluate(input_fn_test)





NameError: name 'model' is not defined

Discriminative rbm calculation

Discriminative rbm calculation

In [ ]:
#setup session variables
n_classes = trY.shape[1]
n = trX.shape[1]
dbn.sizes = tf.placeholder(n,rbm_hb.shape)
n_rbm = enumerate(dbn.sizes)
u = n_rbm

vis_size = dbn.sizes[0]
hid_size = dbn.sizes[1]

#initialize bias and weights for class vector                dbnsetup.m 94
rbm.U = weights(hid_size,n_classes)
rbm.vU = np.zeros(shape=(hid_size,n_classes))

rbm.d = np.zeros(shape=(n_classes,1))
rbm.vd = np.zeros(shape=(n_classes,1))


rbm.W = weights(hid_size,vis_size)





#weight initialization

def weights(m,n):                                                       #dbmsetup.m line 154
    # % initilize weigts from uniform distribution. As described in
    # % Learning Algorithms for the Classification Restricted Boltzmann
    # % machine
    M = max(m,n)
    interval_max = M**(-0.5)
    interval_min = -interval_max
    weights = interval_min + np.multiply(interval_max-interval_min,np.random.uniform(size=(m,n))) 
    
    # assert(max(weights) <= interval_max)
    # assert(min(weights) >= interval_min)
    return weights



[p_y_given_x, F] = rbmpygivenx(rbm,x,'train')


#RBMPYGIVENX calculates class probabilities [p(y|x)]

n_samples = trX.shape[1]          #X parameters (here: 254)         #possibly 1 row and 1 dim. if batch = 1
cwx = (n_w_3*trX[:,i]) + n_hb_3     # -ns       #top layer visible-hidden weights * current entity's features + hidden bias
#matlab n_w_3 =? rbm.W  (rbm.W = visible - hidden weights          -ns

#dropout not used
#rbm.hidden_mask = (rbm.rand(size(n_hidden,opts.batchsize)) > rbm.dropout_hidden);

rbm_vU = w_positive_grad * alpha + momentum * rbm_vW
rbm_U =  rbm_U + rbm_vU

F = tf.placeholder(None)
F = np.transpose(rbm_u,(0, 2, 1)) + cwx       # -ns

class_log_prob = np.zeros(n_samples, n_classes)
for y in range(n_classes):
    class_log_prob[:,y] = sess.run(sum(tf.nn.softplus(F[:,:,y]),1) + rbm_d[y])    # -missing laber layer

# for i in class_log_prob[:]:
#     class_prob[0,i] = (class_log_prob - max(class_log_prob)      

class_prob = np.exp(np.subtract(class_log_prob,max(class_log_prob)))     #  source: rbmpygivenx.m 25 line  class_log_prob[:] - max(class_log_prob)

class_prob = class_prob/sum(class_prob)

[p_y_given_x, F] = class_prob,F                            #  source: rbmdiscriminative.m 49 line

F_sigm = tf.nn.sigmoid(F)

F_sigm_prob = tf.placeholder(F_sigm.shape)


for c in n_classes:
    F_sigm_prob = np.matmul(F_sigm[:,:,c],np.transpose(p_y_given_x[:,c]))
    
#init grads
dw = tf.zeros(n_w_3.shape)
#du = tf.zeros(.shape)             #-ns label - hid weights
dc = tf.zeros(n_hb_3.shape)

class_labels = ey.index(max(ey))             #      -ns declare ey

for c in n_classes:
    bin_idx = class_labels == c
    lin_idx = (c == class_labels)
    
    a = F_sigm[:,lin_idx,c] * x[lin_idx,:]           #    -ns  one-hot matlab vs python index problem
    b = F_sigm_prob[:,:,c]*x
    dw = dw + a-b
    
    #du grad
    du[:,c] = sum(F_sigm[:,bin_idx,c],1) - sum(F_sigm_prob[:,:,c],1)
    #dc grad
    dc = dc + sum(F_sigm[:,bin_idx,c],1) - sum(F_sigm_prob[:,:,c],1)

#dd grad
dd = np.transposesum((ey - p_y_given_x,0))


dw = dw / batchsize
db = tf.zeros(n_vb_3.shape)
dc = dc / batchsize
dd = dd / batchsize
du = du / batchsize
#end of rbmdiscriminative.m





12

In [18]:
n_w_3

array([[-0.01778202, -0.01778202, -0.01778202, ..., -0.01778202,
        -0.01778202, -0.01778202],
       [-0.01778202, -0.01778202, -0.01778202, ..., -0.01778202,
        -0.01778202, -0.01778202],
       [-0.01778202, -0.01778202, -0.01778202, ..., -0.01778202,
        -0.01778202, -0.01778202],
       ...,
       [-0.01778202, -0.01778202, -0.01778202, ..., -0.01778202,
        -0.01778202, -0.01778202],
       [-0.01778202, -0.01778202, -0.01778202, ..., -0.01778202,
        -0.01778202, -0.01778202],
       [-0.01778202, -0.01778202, -0.01778202, ..., -0.01778202,
        -0.01778202, -0.01778202]], dtype=float32)

In [20]:
correct_pred

<tf.Tensor 'Equal_1:0' shape=(?,) dtype=bool>

In [22]:
# predictions = np.zeros(25)
# i = 0
true_preds = 0

for start, end in zip(range(0, len(trX), batchsize), range(batchsize, len(trX), batchsize)):
    batch_x = trX[start:end]
    batch_y = trY[start:end]
    pred = sess.run(correct_pred, feed_dict={X: batch_x, Y: batch_y})
    
    for i in range(len(pred)):
        if pred[i] == True:
            true_preds = true_preds + 1
    # predictions[i] = pred
    # i = i + 1
    
    
    #predictions[i] = pred
    


In [24]:
# true_preds = 0
# for i in range(len(pred)):
#     if pred[i] == True:
#         true_preds = true_preds + 1
        
#true_preds = true_preds
print(true_preds, '/',len(trX))
        

1002 / 2094


In [77]:
predictions = true_preds
predictions

51

In [16]:
#pred.shape
n = trX.shape[1]


254